# Identify driver factors in cell state transitions

The ``find_driver_in_transition`` command identifies key transcription factors that drive changes in gene expression and/or chromatin accessibility during cell state transitions (e.g., differentiation or reprogramming).

You can run this command with:
- expression only,
- accessibility only, or
- both expression and accessibility.

Provide the corresponding input files for the analyses you want to perform.

**Note**: The remaining examples will only show the direct command usage. 

If you need to use Singularity container, please refer to the [`singularity_use.ipynb`](singularity_use.ipynb) tutorial for detailed instructions on using `singularity exec` with `chrombert-tools`.

## Example:

Find driver regulators in fibroblast-to-myoblast transition using both expression and accessibility


In [1]:
import pandas as pd
import numpy as np
import os
workdir="/mnt/Storage2/home/chenqianqian/projects/chrombert_tools/2.test/pull/ChromBERT-tools/examples/cli" # your workdir
os.chdir(workdir)

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]='0'

In [3]:
!chrombert-tools -h

Usage: chrombert-tools [OPTIONS] COMMAND [ARGS]...

  Type -h or --help after any subcommand for more information.

Options:
  -v, --verbose  Verbose logging
  -d, --debug    Post mortem debugging
  -V, --version  Show the version and exit.
  -h, --help     Show this message and exit.

Commands:
  embed_cell_cistrome             Extract cell-specific cistrome...
  embed_cell_gene                 Extract cell-specific gene embeddings
  embed_cell_region               Extract cell-specific region embeddings
  embed_cell_regulator            Extract cell-specific regulator...
  embed_cistrome                  Extract general cistrome embeddings on...
  embed_gene                      Extract general gene embeddings
  embed_region                    Extract general region embeddings
  embed_regulator                 Extract general regulator embeddings on...
  find_context_specific_cofactor  Find context-specific cofactors in...
  find_driver_in_transition       Find driver factors in cell

In [4]:
!chrombert-tools find_driver_in_transition -h

Usage: chrombert-tools find_driver_in_transition [OPTIONS]

  Find driver factors in cell state transitions.

  This tool identifies key transcription factors that drive cell state
  transitions by analyzing changes in gene expression and/or chromatin
  accessibility between two cell states.

  You must provide at least one of the following: - Expression data (--exp-
  tpm1 and --exp-tpm2) - Accessibility data (--acc-peak1, --acc-peak2, --acc-
  signal1, --acc-signal2)

  Providing both expression and accessibility data yields more confident
  results.

Options:
  --exp-tpm1 FILE                 Expression (TPM) file for cell state 1. CSV
                                  format with 'gene_id' and 'tpm' columns.
  --exp-tpm2 FILE                 Expression (TPM) file for cell state 2. CSV
                                  format with 'gene_id' and 'tpm' columns.
  --acc-peak1 FILE                Chromatin accessibility peak BED file for
                                  cell state 1.
 

## Run

In [10]:
# Download data
import subprocess
if not os.path.exists('../data/myoblast_ENCFF647RNC_peak.bed'):
    cmd = f'wget https://www.encodeproject.org/files/ENCFF647RNC/@@download/ENCFF647RNC.bed.gz -O ../data/myoblast_ENCFF647RNC_peak.bed.gz'
    subprocess.run(cmd, shell=True)
    cmd = f"gzip -d ../data/myoblast_ENCFF647RNC_peak.bed.gz"
    subprocess.run(cmd, shell=True)

if not os.path.exists('../data/myoblast_ENCFF149ERN_signal.bigwig'):
    cmd = f'wget https://www.encodeproject.org/files/ENCFF149ERN/@@download/ENCFF149ERN.bigWig -O ../data/myoblast_ENCFF149ERN_signal.bigwig'
    subprocess.run(cmd, shell=True) 


if not os.path.exists('../data/fibroblast_ENCFF184KAM_peak.bed'):
    cmd = f'wget https://www.encodeproject.org/files/ENCFF184KAM/@@download/ENCFF184KAM.bed.gz -O ../data/fibroblast_ENCFF184KAM_peak.bed.gz'
    subprocess.run(cmd, shell=True)
    cmd = f"gzip -d ../data/fibroblast_ENCFF184KAM_peak.bed.gz"
    subprocess.run(cmd, shell=True)


if not os.path.exists('../data/fibroblast_ENCFF361BTT_signal.bigwig'):
    cmd = 'wget https://www.encodeproject.org/files/ENCFF361BTT/@@download/ENCFF361BTT.bigWig -O ../data/fibroblast_ENCFF361BTT_signal.bigwig'
    subprocess.run(cmd, shell=True)

In [11]:
# Runtime estimates:
#   - fast mode: ~3-5 hours
#     (uses all ~19,620 genes for expression analysis, but downsamples 
#      chromatin accessibility regions to 20k for faster training)
#
# Note: Both modes (fast and full) use the complete gene expression dataset. The 'fast' mode 
# only downsamples chromatin accessibility regions, not gene data.

# So this downsampled 5000 genes for expression analysis for test (40-100 minutes)

!chrombert-tools find_driver_in_transition \
  --exp-tpm1 "../data/fibroblast_expression_sample5000.csv" \
  --exp-tpm2 "../data/myoblast_expression_sample5000.csv" \
  --acc-peak1 "../data/fibroblast_ENCFF184KAM_peak.bed" \
  --acc-peak2 "../data/myoblast_ENCFF647RNC_peak.bed" \
  --acc-signal1 "../data/fibroblast_ENCFF361BTT_signal.bigwig" \
  --acc-signal2 "../data/myoblast_ENCFF149ERN_signal.bigwig" \
  --genome 'hg38' \
  --resolution '1kb' \
  --odir output_find_driver_in_transition \
  --direction "2-1" 2> "./tmp/hg38_1kb.stderr.log"

Stage 1: prepare dataset
Processing stage 1: prepare expression dataset
Processing stage 1: prepare chromatin accessibility dataset
Finished Stage 1
Whether to train ChromBERT to predict expression changes in cell state transition: True
Whether to train ChromBERT to predict chromatin accessibility changes in cell state transition: True
Processing stage 2 (exp): train ChromBERT to predict expression changes in cell state transition
Stage 2 (exp): train ChromBERT to predict expression changes in cell state transition

[Attempt 0/2] seed=55
use organisim hg38; max sequence length is 6391
Epoch 0:  20%|████▍                 | 336/1688 [03:34<14:22,  1.57it/s, v_num=0]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0: 100%|████████████████| 105/105 [00:45<00:00,  2.31it/s]
Epoch 0:  40%|▍| 672/1688 [08:19<12:35,  1.34it/s, v_num=0, default_validation/r
Validation: |                                             | 0/? [00:00<?, ?it/s]
Val

In [17]:
odir = "./output_find_driver_in_transition"
exp_odir = f'{odir}/exp'
exp_results_odir = f"{exp_odir}/results"
exp_rank_df = pd.read_csv(os.path.join(exp_results_odir, "factor_importance_rank.csv"))
exp_rank_df

,factors,similarity,rank
0,cbx6,0.942596,1
1,cbx7,0.947722,2
2,cbx8,0.966025,3
3,ring1,0.970225,4
4,chd4,0.972122,5
...,...,...,...
986,irf5,0.996654,987
987,znf157,0.996670,988
988,gtf2i,0.996706,989
989,tfcp2,0.997034,990


In [18]:
acc_odir = f'{odir}/acc'
acc_results_odir = f"{acc_odir}/results"
acc_rank_df = pd.read_csv(os.path.join(acc_results_odir, "factor_importance_rank.csv"))
acc_rank_df

,factors,similarity,rank
0,myog,0.130846,1
1,pax3-foxo1a,0.139224,2
2,myf5,0.158697,3
3,myod1,0.190483,4
4,dux4,0.363659,5
...,...,...,...
986,zbed4,0.983267,987
987,tox4,0.983298,988
988,tfcp2,0.983753,989
989,znf12,0.983788,990


In [21]:
import pandas as pd
merge_odir=f'{odir}/merge'
merge_df = pd.read_csv(os.path.join(merge_odir, "factor_importance_rank.csv"))
merge_df.head(n=10

In [19]:
merge_df = pd.merge(exp_rank_df,acc_rank_df,on='factors',how='inner',suffixes=['_exp','_acc'])
merge_df

,factors,similarity_exp,rank_exp,similarity_acc,rank_acc
0,cbx6,0.942596,1,0.928484,117
1,cbx7,0.947722,2,0.931578,126
2,cbx8,0.966025,3,0.934456,139
3,ring1,0.970225,4,0.929804,119
4,chd4,0.972122,5,0.623431,20
...,...,...,...,...,...
986,irf5,0.996654,987,0.973949,804
987,znf157,0.996670,988,0.980365,961
988,gtf2i,0.996706,989,0.977283,877
989,tfcp2,0.997034,990,0.983753,989


In [20]:
merge_df['total_rank']=((merge_df['rank_exp']+merge_df['rank_acc'])/2).rank().astype(int)
merge_df = merge_df.sort_values('total_rank').reset_index(drop=True)
merge_df

,factors,similarity_exp,rank_exp,similarity_acc,rank_acc,total_rank
0,myf5,0.976115,8,0.158697,3,1
1,neurog2,0.980077,17,0.396391,8,2
2,chd4,0.972122,5,0.623431,20,2
3,myod1,0.981212,22,0.190483,4,4
4,yap1,0.980443,19,0.456161,10,5
...,...,...,...,...,...,...
986,znf26,0.996518,983,0.981732,977,987
987,hes4,0.996514,982,0.982208,979,988
988,zbed4,0.996500,980,0.983267,987,989
989,pitx1,0.996612,986,0.984283,991,990


### Load the fine-tuned checkpoint to infer key regulators and TRN for myoblast (skip fine-tuning)

In [13]:
# if you have already run find_driver_in_transition, you can use the fine-tuned checkpoint to infer drivers
ft_ckpt_exp_dir = "./output_find_driver_in_transition/exp/train/**/*.ckpt"
ft_ckpt_acc_dir = "./output_find_driver_in_transition/acc/train/**/*.ckpt"

import glob
ft_ckpt_exp = glob.glob(ft_ckpt_exp_dir, recursive=True)[0]
ft_ckpt_acc = glob.glob(ft_ckpt_acc_dir, recursive=True)[0]

In [14]:
ft_ckpt_exp,ft_ckpt_acc

('./output_find_driver_in_transition/exp/train/try_00_seed_55/lightning_logs/lightning_logs/version_0/checkpoints/epoch=4-step=129.ckpt',
 './output_find_driver_in_transition/acc/train/try_00_seed_55/lightning_logs/lightning_logs/version_0/checkpoints/epoch=2-step=163.ckpt')

In [16]:
!chrombert-tools find_driver_in_transition \
  --ft-ckpt-exp {ft_ckpt_exp} \
  --ft-ckpt-acc {ft_ckpt_acc} \
  --genome 'hg38' \
  --exp-tpm1 "../data/fibroblast_expression_sample5000.csv" \
  --exp-tpm2 "../data/myoblast_expression_sample5000.csv" \
  --acc-peak1 "../data/fibroblast_ENCFF184KAM_peak.bed" \
  --acc-peak2 "../data/myoblast_ENCFF647RNC_peak.bed" \
  --acc-signal1 "../data/fibroblast_ENCFF361BTT_signal.bigwig" \
  --acc-signal2 "../data/myoblast_ENCFF149ERN_signal.bigwig" \
  --resolution '1kb' \
  --odir output_find_driver_in_transition_load_ckpt \
  --direction "2-1" 2> "./tmp/hg38_1kb.stderr.log"

Stage 1: prepare dataset
Processing stage 1: prepare expression dataset
Processing stage 1: prepare chromatin accessibility dataset
Finished Stage 1
Whether to train ChromBERT to predict expression changes in cell state transition: True
Whether to train ChromBERT to predict chromatin accessibility changes in cell state transition: True
Processing stage 2 (exp): train ChromBERT to predict expression changes in cell state transition
Use fine-tuned ChromBERT checkpoint file: ./output_find_driver_in_transition/exp/train/try_00_seed_55/lightning_logs/lightning_logs/version_0/checkpoints/epoch=4-step=129.ckpt to find driver factors in different expression activity genes
use organisim hg38; max sequence length is 6391
Loading checkpoint from ./output_find_driver_in_transition/exp/train/try_00_seed_55/lightning_logs/lightning_logs/version_0/checkpoints/epoch=4-step=129.ckpt
Loading from pl module, remove prefix 'model.'
Loaded 110/110 parameters
Finished Stage 2 (exp): use fine-tuned ChromBERT

In [26]:
odir = "./output_find_driver_in_transition_load_ckpt"
exp_odir = f'{odir}/exp'
exp_results_odir = f"{exp_odir}/results"
exp_rank_df = pd.read_csv(os.path.join(exp_results_odir, "factor_importance_rank.csv"))
exp_rank_df.head(n=10)

,factors,similarity,rank
0,cbx6,0.942596,1
1,cbx7,0.947722,2
2,cbx8,0.966025,3
3,ring1,0.970225,4
4,chd4,0.972122,5
5,hira,0.974143,6
6,brd7,0.974496,7
7,myf5,0.976115,8
8,klf11,0.977443,9
9,prkdc,0.977606,10


In [28]:
acc_odir = f'{odir}/acc'
acc_results_odir = f"{acc_odir}/results"
acc_rank_df = pd.read_csv(os.path.join(acc_results_odir, "factor_importance_rank.csv"))
acc_rank_df.head(n=10)

,factors,similarity,rank
0,myog,0.130846,1
1,pax3-foxo1a,0.139224,2
2,myf5,0.158697,3
3,myod1,0.190483,4
4,dux4,0.363659,5
5,pax7,0.374296,6
6,tead1,0.387275,7
7,neurog2,0.396391,8
8,six2,0.410954,9
9,yap1,0.456161,10


In [29]:
import pandas as pd
merge_odir=f'{odir}/merge'
merge_df = pd.read_csv(os.path.join(merge_odir, "factor_importance_rank.csv"))
merge_df.head(n=10)

,factors,similarity_exp,rank_exp,similarity_acc,rank_acc,total_rank
0,myf5,0.976115,8,0.158697,3,1
1,neurog2,0.980077,17,0.396391,8,2
2,chd4,0.972122,5,0.623431,20,2
3,myod1,0.981212,22,0.190483,4,4
4,yap1,0.980443,19,0.456161,10,5
5,tcf21,0.981082,21,0.500809,14,6
6,myog,0.986830,40,0.130846,1,7
7,tead1,0.984907,34,0.387275,7,7
8,tbx5,0.985799,36,0.515574,15,9
9,pgbd3,0.984669,33,0.539703,18,9


In [30]:
merge_df = pd.merge(exp_rank_df,acc_rank_df,on='factors',how='inner',suffixes=['_exp','_acc'])
merge_df

,factors,similarity_exp,rank_exp,similarity_acc,rank_acc
0,cbx6,0.942596,1,0.928484,117
1,cbx7,0.947722,2,0.931578,126
2,cbx8,0.966025,3,0.934456,139
3,ring1,0.970225,4,0.929804,119
4,chd4,0.972122,5,0.623431,20
...,...,...,...,...,...
986,irf5,0.996654,987,0.973949,804
987,znf157,0.996670,988,0.980365,961
988,gtf2i,0.996706,989,0.977283,877
989,tfcp2,0.997034,990,0.983753,989


In [31]:
merge_df['total_rank']=((merge_df['rank_exp']+merge_df['rank_acc'])/2).rank().astype(int)
merge_df = merge_df.sort_values('total_rank').reset_index(drop=True)
merge_df

,factors,similarity_exp,rank_exp,similarity_acc,rank_acc,total_rank
0,myf5,0.976115,8,0.158697,3,1
1,neurog2,0.980077,17,0.396391,8,2
2,chd4,0.972122,5,0.623431,20,2
3,myod1,0.981212,22,0.190483,4,4
4,yap1,0.980443,19,0.456161,10,5
...,...,...,...,...,...,...
986,znf26,0.996518,983,0.981732,977,987
987,hes4,0.996514,982,0.982208,979,988
988,zbed4,0.996500,980,0.983267,987,989
989,pitx1,0.996612,986,0.984283,991,990
